# Data Analysis about crimes and violence at São Paulo

#### 1) All datasets about crimes and violenca are from: https://www.ssp.sp.gov.br/estatistica/pesquisa.aspx

#### Informations on datasets since jan-18 until sep-20

#### 2) METADATA of datasets about crimes:

#### Ocorrencia: kind of crime, violence or something like that;
#### Xxx: number of crimes reported on month Xxx (e.g. Dez: number of crimes reported on month December)
#### ANO: year when crime or violence happened
#### BAIRRO: neighborhood or district of police department did the occurrence
#### DP: number of the police department at São Paulo city

#### 3) Dataset about lockdown from: https://www.saopaulo.sp.gov.br/planosp/simi/dados-abertos/

#### 4) To create a geojson file about neighborhoods from São Paulo city you can use this web site: https://geojson.io/

### Libraries

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.offline as py
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = 'browser'

import warnings
warnings.filterwarnings("ignore")
import os
from datetime import *

import glob

import json

### Loading all datasets about crimes and violence:

In [ ]:
# If you will test this notebook just change C:\Users\ricar\Python_Algorithms\VIOLENCIA_SAO_PAULO\datasets\ by the path where datasets are located: 
files = glob.glob(r'C:\Users\ricar\Python_Algorithms\VIOLENCIA_SAO_PAULO\datasets\Mensal-Delegacia-*.csv')
 
array_df = []

for x in files:
    temp_df = pd.read_csv(x, 
                          encoding='iso-8859-1',
                          sep=';')
    array_df.append(temp_df)

df = pd.concat(array_df, axis=0)
#df

df.info()

Looking for missing values:

In [ ]:
df.isnull().sum()

There are missing values because of october, november and december from 2020. Let's replace these missing values by zero:

In [ ]:
df['Out'] = df['Out'].fillna(0)
df['Nov'] = df['Out'].fillna(0)
df['Dez'] = df['Out'].fillna(0)

df.isnull().sum()

Now, there aren't any missing values on dataset.

In [ ]:
# Transform all columns of months to data type int64:

def remove_item(my_list,*args):
    delete = list(args)
    for item in delete:
        while item in my_list:
            my_list.remove(item)
    return my_list


month_list = list(df.columns)
remove_item(month_list,'Ocorrencia', 'ANO', 'DP', 'BAIRRO')

for i in month_list:
    df[i] = df[i].astype('int64')

### Informations about lockdown:

In [ ]:
# loading dataset about lockdown informations:
lockdown = pd.read_csv(r'C:\Users\ricar\Python_Algorithms\VIOLENCIA_SAO_PAULO\datasets\20201105_isolamento.csv', encoding='iso-8859-1',sep=';')


# Looking for Município == SÃO PAULO:
lockdown = lockdown[lockdown['Município1'] == 'SÃO PAULO']

# Creating features MM and PERC_LOCKDOWN:
lockdown[['DS', 'DDMM']] = lockdown['Data'].str.split(',', expand=True)
lockdown[['DD', 'MM']] = lockdown['DDMM'].str.split('/', expand=True)
lockdown['MM'] = lockdown['MM'].astype('int64')
lockdown = lockdown[['MM', 'Média de Índice De Isolamento']]
lockdown['PERC_LOCKDOWN'] = lockdown['Média de Índice De Isolamento'].str.replace('%','').astype('int64')/100
lockdown['ANO'] = 2020


# Final dataset about lockdown:
lockdown = lockdown.groupby(['MM', 'ANO']).agg({'PERC_LOCKDOWN': np.average}).sort_values(by=['MM']).reset_index()
lockdown.rename(columns=({'PERC_LOCKDOWN': 'PERC_LOCKDOWN_AVG'}), inplace = True)
lockdown['PERC_LOCKDOWN_AVG'] = round(lockdown['PERC_LOCKDOWN_AVG'], 2)
#lockdown

# What kind of occurrences would you like to check?

In [ ]:
t = list(pd.unique(df['Ocorrencia']))

occurrences = {}

cont = 1
for occur in t:
    occurrences[cont] = occur
    cont += 1

occurrences

# Make your choice based on values above:

In [ ]:
num_occur = input('Choose some number presented above: ')

if int(num_occur) in range(len(t)+1):
    name_occur = occurrences.get(int(num_occur))
    print("\nLet's start some analysis about", name_occur)
    
else:
    print('\nNumber ' + str(num_occur) + ' is not valid! Try again!')

In [ ]:
#Now, let's take only the occurrence you made your choice:

df1 = df[df['Ocorrencia'].isin([name_occur])]
df1.drop(['Ocorrencia'],axis=1, inplace=True)
df1 = df1[['BAIRRO', 'DP', 'ANO', 'Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']]
#df1



#Applying melt to transform the dataset:

df2 = pd.melt(df1.reset_index(), 
               id_vars=['BAIRRO', 'DP', 'ANO'], 
               var_name='MES', 
               value_name='QTDE').sort_values(['BAIRRO', 'DP', 'ANO'], ascending=False)
#df2



#Now, dropping rows where MES contains string 'index' and creating a new column called MM just to help us to order the dataset:

df2 = df2[df2.MES.str.contains("index") == False].reset_index()
df2.drop('index', axis=1, inplace=True)
df2['MM'] = df2['MES'].map({'Jan': 1, 'Fev': 2, 'Mar': 3, 'Abr': 4, 'Mai': 5, 'Jun': 6, 'Jul': 7, 'Ago': 8, 'Set': 9, 'Out': 10, 'Nov': 11, 'Dez': 12})
#df2

### Creating bar charts about total amount of crime or violence you had choose:

In [ ]:
# ask about period:
mes = input('\n Insert a number from 1 to 12: (P.S.: 1 refers to january and 12 refers to december): ')

if int(mes) not in list(np.unique(df2['MM'])):
    
    print('\n Number ' + str(mes) + ' is not valid! Try again!')

else: 
# creating a dataset with aggregated informations:
    df3 = df2.groupby(['ANO', 'MES', 'MM']).agg({'QTDE': np.sum}).sort_values(by=['ANO', 'MM']).reset_index()
    df3 = pd.merge(df3, lockdown, on=['MM', 'ANO'], how='left').reset_index().drop(['index'],axis=1)
    df3['PERC_LOCKDOWN_AVG'] = df3['PERC_LOCKDOWN_AVG'].fillna(0)
    df3 = df3[df3['MM'] <= int(mes)]

# Creating series:
    y_2018  = df3[df3['ANO']==2018]['QTDE']
    x_2018  = df3[df3['ANO']==2018]['MES']

    y_2019  = df3[df3['ANO']==2019]['QTDE']
    x_2019  = df3[df3['ANO']==2019]['MES']

    y_2020  = df3[df3['ANO']==2020]['QTDE']
    x_2020  = df3[df3['ANO']==2020]['MES']
    
    mpl_y_2020 = df3[df3['ANO']==2020]['PERC_LOCKDOWN_AVG']
    mpl_x_2020 = df3[df3['ANO']==2020]['MES']

# Setting up the kind of chart and setting up series properties:
    year_2018 = go.Bar(x = x_2018,
                       y = y_2018,
                       name = '2018',
                       marker_color = '#EDC824',
                       opacity = .7)


    year_2019 = go.Bar(x = x_2019,
                       y = y_2019,
                       name = '2019',
                       marker_color = '#08AA09',
                       opacity = .5)


    year_2020 = go.Bar(x = x_2020,
                       y = y_2020,
                       name = '2020',
                       marker_color = '#12B6A1',
                       opacity = .5)
    
    mpl_2020 = go.Scatter(x = mpl_x_2020,
                          y = mpl_y_2020,
                          yaxis='y2', 
                          marker_color = '#141461',
                          name = 'Lockdown Average (%)')
    
    y2 = go.layout.YAxis(overlaying='y', side='right', title = 'Lockdown Average (%)')

# Creating a list to storage all settings of chart:
    set_list_year = [year_2018, year_2019, year_2020, mpl_2020]

# Creating Layout:
    layout_year = go.Layout(title='Amount of '+ str(name_occur) + ' by year and month at São Paulo city',
                            yaxis={'title':'Amount of ' + str(name_occur)},
                            yaxis2 = y2,
                            xaxis={'title': 'Month'}
                           )

# Creating the picture will be showed:
    
    #fig = make_subplots(specs=[[{"secondary_y": True}]])
    
    fig = go.Figure(data=set_list_year, layout=layout_year)
    fig.update_yaxes(showgrid=False)
    
# Plotting the chart:
    
    py.iplot(fig)

Calculating Pearson's correlation between the average block percentage and the number of occurrences:

In [ ]:
df3[['PERC_LOCKDOWN_AVG', 'QTDE']].corr()

#### Remeber: "correlation doesn't mean causality."

### Aggregating  informations for Choropleth Map:

In [ ]:
mdm = df2.groupby(['BAIRRO', 'DP', 'ANO']).agg({'QTDE': np.median}).sort_values(by=['ANO', 'QTDE'], ascending=False).reset_index()
mdm.rename(columns=({'QTDE': 'MEDIAN_MONTHLY'}), inplace=True)

ta = df2.groupby(['BAIRRO','DP', 'ANO']).agg({'QTDE': np.sum}).sort_values(by=['ANO', 'QTDE'], ascending=False).reset_index()
ta.rename(columns=({'QTDE': 'TOTAL'}), inplace=True)

mdm_ta = pd.merge(mdm, ta, on=['BAIRRO', 'DP', 'ANO'], how='left').reset_index().drop(['index'],axis=1)
#mdm_ta

### Load geojson file with coordinates for latitude and longitude:

In [ ]:
# loading geojson file:
sp_neighborhoods = json.load(open('delegacias_sp.geojson', 'rb'))

# It shows the keys in geojson file:
sp_neighborhoods['features'][0].keys()

If you chance the number between the square brackets It shows the contents about 'properties'  but it's just some kind of validation:

In [ ]:
sp_neighborhoods['features'][0]['properties']

Creating the dictionary neighborhood_id_map and it will be used on choropleth to look for latitude and longitude coordinates:

In [ ]:
neighborhood_id_map = {}

for feature in sp_neighborhoods['features']:
    feature['id'] = feature['properties']['DP']
    neighborhood_id_map[feature['properties']['BAIRRO']] = feature['id']
    
# print(neighborhood_id_map)

# Choropleth Map of neighborhoods from São Paulo:

In [ ]:
print('There are these years on dataset:', np.unique(mdm_ta['ANO']))

### Choropleth Map about total cases of crime, violence or something like that:

In [ ]:
# Message about input of year:
year = input('\n Insert a year from the list above: ')

if int(year) in np.unique(mdm_ta['ANO']):
    
# filter from year and create the object with series:
    mdm_ta_year = mdm_ta[mdm_ta['ANO'] == int(year)]


# create the choropleth map:
    fig = px.choropleth(mdm_ta_year, 
                        locations='DP', 
                        geojson=sp_neighborhoods, 
                        color='TOTAL',
                        color_continuous_scale="bupu",
                        hover_name='BAIRRO',
                        hover_data=['TOTAL'],
                        )
    fig.update_geos(fitbounds='locations', visible = False)
    fig.update_layout(title_text='Amount of ' + str(name_occur) + ' by neighborhoods from São Paulo - ' + str(year), 
                      legend_title = 'TOTAL')

    fig.show()
    
else:
    
    print("\n Year isn't between 2018 and 2020. Please try again!")